# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Применение-алгоритмов-машинного-обучения-для-задачи-банковского-скоринга" data-toc-modified-id="Применение-алгоритмов-машинного-обучения-для-задачи-банковского-скоринга-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Применение алгоритмов машинного обучения для задачи банковского скоринга</a></div>

# Применение алгоритмов машинного обучения для задачи банковского скоринга

Входными данными являются наборы для обучения и для тестирования:
* bank_scoring_train.csv.gz
* bank_scoring_test.csv.gz

Описание полей:
* SeriousDlqin2yrs - Флаг выхода на просрочку через 90 дней
* RevolvingUtilizationOfUnsecuredLines - Общий баланс по кредитным картам и личным кредитным линиям, за исключением недвижимости, и без задолженности по кредитам, таким как автокредиты, деленные на сумму кредитных лимитов
* age - Возраст заёмщика в годах
* NumberOfTime30-59DaysPastDueNotWorse - Сколько раз заемщик просрочил выплату не более, чем на 30-59 дней, за последние 2 года.
* DebtRatio - Ежемесячные платежи по долгам, алименты и прожиточные расходы, делённые на месячный валовый доход
* MonthlyIncome - Ежемесячный доход
* NumberOfOpenCreditLinesAndLoans - Количество открытых кредитов (например, автокредит или закладная, кредитные карты и т.д.)
* NumberOfTimes90DaysLate - Сколько раз заёмщик просрочил выплату на 90 дней и более.
* NumberRealEstateLoansOrLines - Количество ипотечных кредитов и кредитов на недвижимость
* NumberOfTime60-89DaysPastDueNotWorse - Сколько раз заёмщик просрочил выплату не более, чем на 60-89 дней, за последние 2 года.
* NumberOfDependents - Количество иждивенцев в семье, исключая самого заёмщика (супруга, дети и т.д.)

Целевой переменной является SeriousDlqin2yrs

** Рекомендации по использованию моделей, метрик и кросс-валидации представлены в compare_algorithms_emp **

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
data1 = pd.read_csv("bank_scoring_train.csv", sep='\t')

In [4]:
data1.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,0,0.975614,39,1,0.678552,1850.0,4,0,0,0,3.0
1,0,0.006634,75,0,0.082087,10500.0,21,0,1,0,0.0
2,0,0.227432,48,1,0.202145,6059.0,9,0,1,0,3.0
3,0,0.290217,59,0,0.674392,4400.0,6,0,2,0,0.0
4,0,0.000000,80,0,0.000000,2596.0,2,0,0,0,1.0


In [5]:
X = data1.drop(['SeriousDlqin2yrs'], axis=1)
y = data1['SeriousDlqin2yrs']

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)
print (X_train.shape)
print (X_test.shape)

(64464, 10)
(31752, 10)


In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [9]:
log_regression = LogisticRegression(penalty='l2',class_weight='balanced')
log_regression.fit(X_train, y_train)
pred_train = log_regression.predict_proba(X_train)[:,1]
prediction = log_regression.predict_proba(X_test)[:,1]
log_regression.coef_

array([[ -3.48166512e-05,  -2.74079040e-02,   5.42373858e-01,
         -1.01271588e-04,  -3.53645996e-05,   2.93538422e-03,
          5.34222081e-01,   1.04624679e-01,   7.74653139e-02,
          8.61587481e-02]])

In [10]:
from sklearn import metrics 

In [11]:
metrics.roc_auc_score(y_train, pred_train), metrics.roc_auc_score(y_test, prediction)

(0.7852564068621255, 0.79718581731202298)

In [12]:
data2 = pd.read_csv("bank_scoring_test.csv", sep='\t')
X_test_end = data2.drop(['SeriousDlqin2yrs'], axis=1)
y_test_end = data2['SeriousDlqin2yrs']

In [13]:
prediction_end = log_regression.predict(X_test_end)
prediction_proba_end = log_regression.predict_proba(X_test_end)

In [14]:
print(metrics.classification_report(y_test_end, prediction_end, digits=4))
print('roc_auc', metrics.roc_auc_score(y_test_end, prediction_end), metrics.roc_auc_score(y_test_end, prediction_proba_end[:,1]))
print('log_loss', metrics.log_loss(y_test_end, prediction_proba_end))

             precision    recall  f1-score   support

          0     0.9671    0.7945    0.8724     22407
          1     0.1843    0.6318    0.2853      1646

avg / total     0.9135    0.7834    0.8322     24053

roc_auc 0.713181623235 0.787037644079
log_loss 0.580186799558


In [15]:
def pred(clf):
    clf.fit(X_train, y_train)
    
    pred_train = clf.predict(X_train)
    prediction = clf.predict(X_test)
    
    pred_p_train = clf.predict_proba(X_train)[:,1]
    prediction_p = clf.predict_proba(X_test)[:,1]
    print('Train_file')
    print(metrics.classification_report(y_train, pred_train, digits=4), metrics.classification_report(y_test, prediction, digits=4))
    print(metrics.roc_auc_score(y_train, pred_p_train), metrics.roc_auc_score(y_test, prediction_p))

    prediction_end = clf.predict(X_test_end)
    prediction_proba_end = clf.predict_proba(X_test_end)
    print('\nTest_file')
    print(metrics.classification_report(y_test_end, prediction_end, digits=4))
    print('roc_auc', metrics.roc_auc_score(y_test_end, prediction_end), metrics.roc_auc_score(y_test_end, prediction_proba_end[:,1]))
    print('log_loss', metrics.log_loss(y_test_end, prediction_proba_end))

In [33]:
k_neighbors = KNeighborsClassifier(n_neighbors=5)
pred(k_neighbors)

Train_file
             precision    recall  f1-score   support

          0     0.9433    0.9951    0.9685     60011
          1     0.7446    0.1938    0.3076      4453

avg / total     0.9296    0.9397    0.9228     64464
              precision    recall  f1-score   support

          0     0.9354    0.9903    0.9621     29581
          1     0.3417    0.0686    0.1143      2171

avg / total     0.8948    0.9273    0.9041     31752

0.956616852073 0.721409907897

Test_file
             precision    recall  f1-score   support

          0     0.9357    0.9911    0.9626     22407
          1     0.3769    0.0735    0.1230      1646

avg / total     0.8975    0.9283    0.9052     24053

roc_auc 0.532292880507 0.712088960548
log_loss 1.09147405232


In [16]:
decision_tree = DecisionTreeClassifier(max_depth=7, class_weight='balanced')
pred(decision_tree)

Train_file
             precision    recall  f1-score   support

          0     0.9811    0.7627    0.8582     60011
          1     0.2005    0.8017    0.3207      4453

avg / total     0.9272    0.7654    0.8211     64464
              precision    recall  f1-score   support

          0     0.9782    0.7529    0.8509     29581
          1     0.1865    0.7715    0.3003      2171

avg / total     0.9241    0.7542    0.8133     31752

0.860048283011 0.832611799334

Test_file
             precision    recall  f1-score   support

          0     0.9806    0.7593    0.8559     22407
          1     0.1954    0.7959    0.3138      1646

avg / total     0.9269    0.7618    0.8188     24053

roc_auc 0.777592528936 0.838059551235
log_loss 0.495705938517


In [18]:
import sklearn.ensemble

In [23]:
clf_rf_10 = sklearn.ensemble.RandomForestClassifier(n_estimators=10, min_samples_split=8, min_samples_leaf=4, max_features=10, random_state=0)
clf_rf_100 = sklearn.ensemble.RandomForestClassifier(n_estimators=100, min_samples_split=8, min_samples_leaf=4, max_features=10, random_state=0)
clf_rf_1000 = sklearn.ensemble.RandomForestClassifier(n_estimators=1000, min_samples_split=8, min_samples_leaf=4, max_features=10, random_state=0)


clf_rf_10.fit(X_train, y_train)
clf_rf_100.fit(X_train, y_train)
clf_rf_1000.fit(X_train, y_train)

pred_10 = clf_rf_10.predict_proba(X_test_end)
pred_100 = clf_rf_100.predict_proba(X_test_end)
pred_1000 = clf_rf_1000.predict_proba(X_test_end)

score_10 = sklearn.metrics.log_loss(y_test_end, pred_10)
score_100 = sklearn.metrics.log_loss(y_test_end, pred_100)
score_1000 = sklearn.metrics.log_loss(y_test_end, pred_1000)

print('Score Random Forest n_trees=10: %s' % (score_10))
print('Score Random Forest n_trees=100: %s' % (score_100))
print('Score Random Forest n_trees=1000: %s' % (score_1000))

Score Random Forest n_trees=10: 0.316431706051
Score Random Forest n_trees=100: 0.178251775264
Score Random Forest n_trees=1000: 0.163012250104


In [24]:
clf_gbm = sklearn.ensemble.GradientBoostingClassifier(learning_rate=0.03, n_estimators=100, subsample=0.9, min_samples_split=8, min_samples_leaf=4, max_depth=3, max_features=8, random_state=0)

clf_gbm.fit(X_train, y_train)

pred_gbm = clf_gbm.predict_proba(X_test_end)

score_gbm = sklearn.metrics.log_loss(y_test_end, pred_gbm)

print('Score Gradient Boosting: %s' % (score_gbm))

Score Gradient Boosting: 0.188104710037
